# COGS 108 - Final Project 

# Overview

*Fill in your overview here*

# Names

- Amanda Chung
- Christine Fann
- Darren Liu
- Kylee Peng
- Priscila Romero

# Group Members IDs

- A########
- A########
- A15040930
- A########
- A########

# Research Question

What is the relationship between a college major and post-graduate success?
- How does graduating with a certain major impact one's future career in pursuing further education or job opportunities?
- What is our call to action based on the median salary of majors with lower popularity?
- What is the male to female ratio for popular majors?


## Background and Prior Work

*Fill in your background and prior work here* 

References (include links):
- 1)
- 2)

# Hypothesis


*Fill in your hypotheses here*

# Dataset(s)

All our datasets are from https://github.com/fivethirtyeight/data/tree/master/college-majors

(Copy this information for each dataset)
- Dataset Name:
- Link to the dataset:
- Number of observations:

1-2 sentences describing each dataset. 

If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.

# Setup

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.stats import linregress

In [19]:
# read datasets
all_ages = pd.read_csv('data/all-ages.csv')
grad_students = pd.read_csv('data/grad-students.csv')
recent_grads = pd.read_csv('data/recent-grads.csv')
women_stem = pd.read_csv('data/women-stem.csv')

# Data Cleaning

### Extract wanted columns

In [27]:
wanted_cols = 'Rank Major Major_category Employed Full_time Part_time Median P25th P75th Total'.split()
rec_wanted = recent_grads.copy()[wanted_cols]
rec_wanted.head()

,Rank,Major,Major_category,Employed,Full_time,Part_time,Median,P25th,P75th,Total
0,1,PETROLEUM ENGINEERING,Engineering,1976,1849,270,110000,95000,125000,2339.0
1,2,MINING AND MINERAL ENGINEERING,Engineering,640,556,170,75000,55000,90000,756.0
2,3,METALLURGICAL ENGINEERING,Engineering,648,558,133,73000,50000,105000,856.0
3,4,NAVAL ARCHITECTURE AND MARINE ENGINEERING,Engineering,758,1069,150,70000,43000,80000,1258.0
4,5,CHEMICAL ENGINEERING,Engineering,25694,23170,5180,65000,50000,75000,32260.0


### Feature Engineering
Changing the following columns:
- Employed -> Employment_rate
- Full_time -> Full_time_rate
- Part_time -> Part_time_rate

Creating the following columns
- Unemployment_rate

In [28]:
# Employment rate
rec_wanted['Employment_rate'] = rec_wanted['Employed'] / rec_wanted['Total']

# Unemployment_rate
rec_wanted['Unemployment_rate'] = 1 - rec_wanted['Employment_rate']

# Full time rate
rec_wanted['Full_time_rate'] = rec_wanted['Full_time'] / rec_wanted['Total']

# Part time rate 
rec_wanted['Part_time_rate'] = rec_wanted['Part_time'] / rec_wanted['Total']

rec_wanted = rec_wanted.drop('Employed Full_time Part_time'.split(), axis=1)
rec_wanted.head()

,Rank,Major,Major_category,Median,P25th,P75th,Total,Employment_rate,Unemployment_rate,Full_time_rate,Part_time_rate
0,1,PETROLEUM ENGINEERING,Engineering,110000,95000,125000,2339.0,0.844805,0.155195,0.790509,0.115434
1,2,MINING AND MINERAL ENGINEERING,Engineering,75000,55000,90000,756.0,0.846561,0.153439,0.735450,0.224868
2,3,METALLURGICAL ENGINEERING,Engineering,73000,50000,105000,856.0,0.757009,0.242991,0.651869,0.155374
3,4,NAVAL ARCHITECTURE AND MARINE ENGINEERING,Engineering,70000,43000,80000,1258.0,0.602544,0.397456,0.849762,0.119237
4,5,CHEMICAL ENGINEERING,Engineering,65000,50000,75000,32260.0,0.796466,0.203534,0.718227,0.160570


# Data Analysis & Results

# Ethics & Privacy

*Fill in your ethics & privacy discussion here*

# Conclusion & Discussion

*Fill in your discussion information here*